<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Two: Exploratory data analysis and feature extraction. </h2>	


<h5> In this section, Compute at least 15 such image features (a method for each), including the following (NOTE: At least 10 of these must be scalar features and 2 matrix-based features): (i) image size, (ii) average of the red-channel intensity, (iii) aspectratio. This will require significant explatoratory research and data analysis. The first one is already implemented for you, and the next two are pre-specified. Additional requirements specfied in pdf. </h5>

In [1]:
%store -r train
%store -r test

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from numpy.linalg import matrix_rank

In [30]:
# Returns the pixel size of the image
def ft0(image):
    return image.size

# Returns the average of the red-channel pictures for the images
def ft1(image):
    return np.mean(image, axis=(0, 1))[0]

#Returns the aspect ratio of the image
def ft2(image):
    return image.shape[0] / image.shape[1]

# Returns the average of the green-channel pictures for the images
def ft3(image):
    mean = np.mean(image, axis=(0, 1))
    if mean.shape[0] >= 2:
        return mean[1]
    else:
        return 0

# Returns the average of the blue-channel pictures for the images
def ft4(image):
    mean = np.mean(image, axis=(0, 1))
    if mean.shape[0] >= 3:
        return mean[2]
    else:
        return 0
    
# Returns the cornerHarris proportion
def ft5(image):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)
    return (dst>0.01 * dst.max()).sum() / tmp.size

# Returns the cornerHarris rank divided by max possible rank
def ft6(image):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    gray = np.float32(gray)
    dst = cv2.cornerHarris(gray,2,3,0.04)
    return matrix_rank(dst) / min(dst.shape)

In [3]:
img = train['Pictures'][10]
ft6(img)

NameError: name 'ft6' is not defined

In [41]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd

X = pd.DataFrame()


img = train['Pictures'][10]
img = np.copy(img)
gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

sift = cv2.xfeatures2d.SURF_create()


kp,des = sift.detectAndCompute(gray, None)



X['feature']=np.array([des,des])
X['bias'] = np.array([1,2])

#X=np.array([[1,2],[3,4]])
y=np.array([1,2])

#from sklearn.linear_model import LogisticRegression
#mod = LogisticRegression().fit(X,y)


Exception: Data must be 1-dimensional

 We expect all external sources sited, and significant indication of EDA. 

<h4> Graphs </h4>

<h4> Sources </h4>

<h4> DataFrame Creation </h4>

In [ ]:
def feature_frame(df):
    return df
    #Returns data-frame with all the features now inside, and calculated


In [ ]:
feature_frame(data_from_nb1)